# **🔍 Project Title**

# 🔧 Fine-Tuning DeepSeek Coder 1.3B for Lawyer Chatbot using LoRA


# **🛠️ Environment Setup**

## ✅ Step 0: Install Required Libraries


In [1]:
!pip install -U bitsandbytes
!pip install -U transformers accelerate peft


# **📦 Load Pretrained Model**

## ✅ Step 1: Load Model & Tokenizer with 8bit Quantization

In [2]:

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model = "deepseek-ai/deepseek-coder-1.3b-instruct"

quant_config = BitsAndBytesConfig(load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto",
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

# **🎯 Apply LoRA Adapter**

## ✅ Step 2: Apply LoRA

In [3]:

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,572,864 || all params: 1,348,044,800 || trainable%: 0.1167


# **📁 Load and Format Dataset**

## ✅ Step 3: Load Dataset

In [4]:
from google.colab import files
uploaded = files.upload()


Saving lawyer_data.jsonl to lawyer_data.jsonl


In [5]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="lawyer_data.jsonl")

def format_example(example):
    return {
        "text": f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    }

dataset = dataset.map(format_example)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

# **✂️ Tokenization**

## ✅ Step 4: Tokenize

In [6]:

def tokenize(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n"
    response = example["output"]

    input_enc = tokenizer(prompt + response, truncation=True, padding="max_length", max_length=512)
    label_enc = tokenizer(response, truncation=True, padding="max_length", max_length=512)

    label_enc["input_ids"] = [
        token if token != tokenizer.pad_token_id else -100 for token in label_enc["input_ids"]
    ]

    return {
        "input_ids": input_enc["input_ids"],
        "attention_mask": input_enc["attention_mask"],
        "labels": label_enc["input_ids"]
    }

dataset = dataset.map(tokenize, remove_columns=dataset["train"].column_names)


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [9]:
from transformers import AutoConfig

# Load base model's config
base_model = "deepseek-ai/deepseek-coder-1.3b-instruct"
config = AutoConfig.from_pretrained(base_model)

# Save it inside your fine-tuned model folder
config.save_pretrained("/content/deepseek_lawyer")  # or your path


# **🧠 Prepare Trainer**

## ✅ Step 5: Set Training Arguments

In [10]:

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="deepseek_lawyer",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer
)


/tmp/ipython-input-1122192795.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


# **🚀 Train the Model**

## ✅ Step 6:  Train Model using LoRA

In [11]:
trainer.train()

Step,Training Loss
10,4.371900
20,4.459000
30,3.800800
40,3.333100
50,3.149200
60,2.692200
70,2.533100
80,2.382400
90,2.051600
100,2.006900


TrainOutput(global_step=750, training_loss=0.6176725788911184, metrics={'train_runtime': 420.1323, 'train_samples_per_second': 3.57, 'train_steps_per_second': 1.785, 'total_flos': 5907384631296000.0, 'train_loss': 0.6176725788911184, 'epoch': 3.0})

# **💾 Save the Model**

## ✅ Step 7: Save Fine-Tuned Model

In [14]:
model.save_pretrained("/content/drive/MyDrive/deepseek_lawyer")
tokenizer.save_pretrained("/content/drive/MyDrive/deepseek_lawyer")
config.save_pretrained("/content/drive/MyDrive/deepseek_lawyer")

# **🧪 Inference / Test Chatbot**

## ✅ Step 8: Run Inference

In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

# Load base model from Hugging Face
base_model = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

bnb_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=bnb_config, device_map="auto", trust_remote_code=True)

# Load your LoRA adapter
model = PeftModel.from_pretrained(model, "/content/drive/MyDrive/deepseek_lawyer")

# Inference function
def ask(question):
    prompt = f"### Instruction:\n{question}\n### Response:\n"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = result.replace(prompt, "").strip()
    print("👨‍⚖️ Lawyer Assistant:\n", response)

# Example
ask("What is law?")


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:457: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


👨‍⚖️ Lawyer Assistant:
 Law is the formal agreement between two parties for a particular transaction. It requires payment and carries the terms of the contract.


In [16]:
ask("what is criminal law?")

👨‍⚖️ Lawyer Assistant:
 Criminal law is the law in case of a crime. It defines the terms of the crime and the payment depending on the case.


In [17]:
ask("what is FIR?")

👨‍⚖️ Lawyer Assistant:
 FIR is short for Field-Interpolation. It is a method of interpolating missing data values in digital storage. It is commonly used in digital storage systems.
